# 5 useful functions for your Time Series projects in Data Science

## Libraries, Constants and dataset

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

The dataset comes from the SNCF Open Data. It shows the availability of trains over the next 30 days. This dataset is updated every morning.

In [2]:
# Downloading dataset
!curl --silent -o data1.csv https://ressources.data.sncf.com/explore/dataset/tgvmax/download/?format=csv

# Loading dataset
df = (pd
      .read_csv('data1.csv', delimiter=';')
      .assign(date=lambda d: pd.to_datetime(d['date']),
              travel_duration=lambda d: (pd.to_datetime(d['heure_arrivee']) - pd.to_datetime(d['heure_depart'])).astype('timedelta64[m]'))
      .rename(columns={'origine': 'departure', 'destination': 'arrival'})
      [['date', 'departure', 'arrival', 'travel_duration']]
      )

df.head(5)

,date,departure,arrival,travel_duration
0,2022-09-07,NIMES,NARBONNE,87.0
1,2022-09-07,NIMES,TOULOUSE MATABIAU,165.0
2,2022-09-07,LILLE EUROPE,LYON (gares intramuros),177.0
3,2022-09-07,LILLE EUROPE,AEROPORT CDG2 TGV ROISSY,48.0
4,2022-09-07,AEROPORT CDG2 TGV ROISSY,LYON (gares intramuros),123.0


We aggregate by origin x arrival and calculate two metrics: the number of available trains and the aggregated travel time

In [3]:
df = (df
      .groupby(['date', 'departure', 'arrival'], as_index=False)
      .agg(n_train=('date', 'count'),
           travel_duration=('travel_duration', 'sum'))
      )

We study only the 20 most popular originating stations

In [4]:
top_departure = (df
                 .groupby('departure').n_train.sum()
                 .sort_values(ascending=False).index[0:20]
                 )
df = df.query('departure in @top_departure').reset_index(drop=True)

## Function 1: id_time_coverage()

In [5]:
def id_time_coverage(data, y, time_var, args=dict()):
    """Displays the time coverage for each ID.

    Parameters
    ----------
    data (pd.DataFrame): Table with at least one identifier and one time variable.
    y (list): List representing the identifying variable. Can be a combination of
    multiple ID.
    time_var (string): Name of the time variable.
    args (dict): Other arguments to the px.scatter function.

    Returns
    -------
    plotly.graph_objs._figure.Figure
    """
    data = (data
            .assign(id = lambda d: d[y].agg(' - '.join, axis=1))
            .groupby(['id', time_var], as_index=False)
            .sum())

    fig = px.scatter(data, y='id', x=time_var, **args, height=100+25*len(data['id'].unique()))
    fig.update_yaxes(categoryorder='array', categoryarray= data['id'].unique())

    return fig

In [6]:
id_time_coverage(data=df, y=['departure'], time_var='date',
                 args={'color': 'n_train', 'size': 'n_train'})

In [7]:
top_arrival = (df
               .groupby('arrival').n_train.sum()
               .sort_values(ascending=False).index[0:5]
               )

id_time_coverage(data=df.query('arrival in @top_arrival'),
                 y=['departure', 'arrival'], time_var='date',
                 args={'color': 'n_train', 'size': 'n_train'})

## Function 2: id_importance() 

In [8]:
def id_importance(data, id_vars, cumul_var, type='tab'):
    """Returns the cumulative importance table or graph of the identifiers.

    Parameters
    ----------
    data (pd.DataFrame): Table with at least one identifier and one variable to
    be aggregated.
    id_vars(list): List representing the identifying variable. Can be a
    combination of multiple ID.
    cumul_var (string): Name of the variable to be aggregated.
    type (string): Either 'tab' or 'graph'.

    Returns
    -------
    If type == 'tab', returns a pd.DataFrame
    If type == "graph", returns a plotly.graph_objs._figure.Figure
    """
    tab = (
        data
        .groupby(id_vars, as_index=False)
        .agg(sum=(cumul_var, 'sum'))
        .assign(pct=lambda d: (d['sum']/sum(d['sum'])).round(8))
        .sort_values('pct', ascending=False, ignore_index=True)
        .assign(cumsum_pct=lambda d: d['pct'].cumsum())
        .rename(columns={'cumsum_pct': 'cumulative sum' + cumul_var})
        )

    if type == "graph":
        tab = (tab
               .reset_index()
               .rename(columns={'index': 'Combination' + str(id_vars)}))
        return px.line(tab, y='cumulative sum' + cumul_var,
                       x='Combination' + str(id_vars))
    else:
        return tab

In [9]:
id_importance(df, ['departure', 'arrival'], 'n_train', type='graph')

## Function 3: id_cross_importance ()

In [10]:
def id_cross_importance(data, id1, id2, weight_var, x_var='pct', title=''):
    """Displays the distribution of crosses between different identifying
    variables.

    Parameters
    ----------
    data (pd.DataFrame): Table with at least two identifiers and one variable
    to be aggregated.
    id1(list): List representing the first indentifying variable. Can be a
    combination of multiple ID.
    id(lis2t): List representing the second indentifying variable. Can be a
    combination of multiple ID.
    weight_var (string): Name of the variable to be aggregated.
    x_var (string): Either 'pct' or 'val'. Allow to display the distribution in
    ('pct') percentage or in ('val') value.
    title (string): Title of the graph.

    Returns
    -------
    plotly.graph_objs._figure.Figure
    """

    tab = (data
           .groupby(id1 + id2, as_index=False)
           .agg(val=(weight_var, 'sum'))
           .assign(id1=lambda d: d[id1].agg(' - '.join, axis=1),
                   id2=lambda d: d[id2].agg(' - '.join, axis=1))
           )

    order_airport = (tab
                     .groupby('id1').agg(total=('val', 'sum'))
                     .sort_values('total', ascending=True).index.tolist())

    tab = (
        tab
        .merge(tab
               .groupby('id1', as_index=False)
               .agg(total=('val', 'sum')), on='id1')
        .assign(pct=lambda d: d['val']/d['total'])
        .sort_values(['id1', 'pct'], ascending=False, ignore_index=True)
        .groupby('id1', as_index=False)
        .apply(lambda d: d.reset_index(drop=True)).reset_index()
        .assign(top=lambda d: d['level_1'].astype('category'))
        .drop(['level_0', 'level_1'], axis=1)
        .assign(pct_str=lambda d: (100*d['pct']).round(1).astype('str') + '%')
        )

    fig = px.bar(tab, y='id1',  x=x_var,
                 color='top', text='id2',
                 title=title,
                 height=100+40*len(order_airport),
                 custom_data=['id2', 'pct_str', 'top'])

    fig.update_traces(hovertemplate="<br>".join([str(id2)+": %{customdata[0]}",
                                                 "pct: %{customdata[1]}",
                                                 "top: %{customdata[2]}"]))

    fig.update_yaxes(categoryorder='array', categoryarray=order_airport)

    return fig

In [11]:
id_cross_importance(data=df,
                    id1=['departure'], id2=['arrival'],
                    weight_var='n_train', x_var='pct',
                    title='Cross id representation').show()

With the parameter **x_var**, can be displayed in value instead of percentage

In [12]:
id_cross_importance(data=df,
                    id1=['departure'], id2=['arrival'],
                    weight_var='n_train', x_var='val',
                    title='Cross id representation').show()

## Function 4: ts_features() 

In [13]:
def ts_lag(data, id_vars, time_var, lagged_vars, period, n_period):
    """Create a table with the lags of one or multiple variable by ID. No need
    of a complete dataset.

    Parameters
    ----------
    data (pd.DataFrame): Table with at least one identifier, one time variable
    and the variable to be lagged. Must be sorted.
    id_vars(list): List representing by which variables we want to compute the
    lags.
    time_var(string): Time variable.
    period (string): Type of period, such as 'D', 'W'.
    n_period (int): Number of periods.

    Returns
    -------
    pd.DataFrame
    """

    period_str = str(n_period) + period

    dict_rename = {time_var: 'to_join'}
    for var in lagged_vars:
        dict_rename[var] = f'lag_{period_str}_{var}'

    df_lag = (
        data[id_vars + [time_var] + lagged_vars]
        .rename(columns=dict_rename)
        .assign(to_join=lambda d: d['to_join'] +
                pd.to_timedelta(n_period, period))
    )

    return df_lag

In [14]:
ts_lag(df, ['departure', 'arrival'], time_var='date',
       lagged_vars=['n_train', 'travel_duration'], period='W', n_period=1)

,departure,arrival,to_join,lag_1W_n_train,lag_1W_travel_duration
0,AVIGNON TGV,AEROPORT CDG2 TGV ROISSY,2022-08-24,3,587.0
1,AVIGNON TGV,AIX EN PROVENCE TGV,2022-08-24,19,373.0
2,AVIGNON TGV,ANTIBES,2022-08-24,6,1080.0
3,AVIGNON TGV,BEAUNE,2022-08-24,1,159.0
4,AVIGNON TGV,BELFORT MONTBELIARD TGV,2022-08-24,3,722.0
...,...,...,...,...,...
16291,VALENCE TGV RHONE ALPES SUD,ST PIERRE DES CORPS,2022-09-21,1,228.0
16292,VALENCE TGV RHONE ALPES SUD,ST RAPHAEL VALESCURE,2022-09-21,1,182.0
16293,VALENCE TGV RHONE ALPES SUD,STRASBOURG,2022-09-21,1,273.0
16294,VALENCE TGV RHONE ALPES SUD,TOULON,2022-09-21,1,125.0


## Function 5: ts_visualisation() 

In [15]:
def ts_visualisation(data, list_id, group, x_var, y_var, col_var,
                     threshold_train=None, weekdays=False, scatter=False):
    """Display one or multiple time series with an ID selector

    Parameters
    ----------
    data (pd.DataFrame): Table with at least one identifier, one time variable
    and the variable to display.
    list_id(list): List of ID to be included in the selector.
    group(string): Name of the ID variable.
    x_var (string): Time variable.
    y_var (list): List of variable to be displayed.
    col_var (list): List of colors corresponding to each curve.
    threshold_train (string): OPTIONAL. Date of the train threshold.
    weekdays (bool): Default = False. If True, illustrate the weekdays.
    scatter (bool): Default = False. If True, display a scatter plot as well as
    line plot.

    Returns
    -------
    plotly.graph_objs._figure.Figure
    """

    figs = dict()
    params = []
    final_fig = make_subplots(specs=[[{"secondary_y": True}]])

    for i in range(0, len(list_id)):
        id = list_id[i]
        df_id = data.loc[data[group] == id]

        figs[id] = {'final': make_subplots(specs=[[{"secondary_y": True}]]),
                    'actuals': px.line(df_id, x=x_var, y=y_var,
                                       color_discrete_sequence=col_var)}

        figs[id]['actuals'].update_xaxes(tickformat="%a %d-%m")

        figs[id]['final'].add_traces(data=figs[id]['actuals'].data)

        if scatter:
            figs[id]['actuals_scatter'] = (
              px.scatter(df_id, x=x_var, y=y_var,
                         color_discrete_sequence=col_var))
            figs[id]['final'].add_traces(data=figs[id]['actuals_scatter'].data)
        if weekdays:
            df_id = df_id.assign(weekday_name=lambda d: d[x_var].dt.day_name())
            figs[id]['weekdays'] = px.scatter(df_id, x=x_var, y=y_var,
                                              color='weekday_name')

            marker_type = dict(size=8, line=dict(width=1,  color='black'))
            selector_type = dict(mode='markers')
            figs[id]['weekdays'].update_traces(marker=marker_type,
                                               selector=selector_type)

            figs[id]['final'].add_traces(data=figs[id]['weekdays'].data)

        final_fig.add_traces(data=figs[id]['final'].data)

        list_show = [False] * len(list_id)
        list_show[i] = True
        param = {'args': [{'visible': np.repeat([list_show],
                                                len(figs[id]['final'].data))},
                          {'showlegend' : True,
                          'title': f'id = {id}'}],
                'label': id,
                'method': 'update'}
        params.append(param)

    if threshold_train is not None:
        final_fig.add_vrect(x0=threshold_train, x1=data[x_var].max(),
                            fillcolor="LightSeaGreen", layer="below",
                            line_width=0)

    final_fig.update_layout(updatemenus=[go.layout.Updatemenu(active=-1,
                                                              type="dropdown",
                                                              buttons=params)])

    return final_fig

We create fake predictions to illustrate this function.

In [16]:
# Assuming a train threshold
THRESHOLD_TRAIN = df['date'].max() - pd.to_timedelta(12, 'D')
THRESHOLD_FORECAST = df['date'].max() - pd.to_timedelta(7, 'D')

df = (
    df
    .assign(n_train_pred=lambda d: d['n_train'] +
            d['n_train']*pd.Series(np.random.uniform(-0.7, 0.7, len(df))),
            n_train=lambda d: d['n_train'].mask(d['date'] > THRESHOLD_FORECAST,
                                                np.nan)))

In [17]:
ts_visualisation(data=df.groupby(['departure', 'date'],
                                 as_index=False).sum().replace(0, np.nan),
                 list_id=df.departure.unique(),
                 group='departure',
                 x_var='date',
                 y_var=['n_train', 'n_train_pred'],
                 col_var=['black', 'red'],
                 threshold_train=THRESHOLD_TRAIN,
                 weekdays=True, scatter=False)

By specifying an ID, we can also plot time series by combination of multiple ID variable

In [18]:
df = (df
      .assign(id=lambda d: d[['departure', 'arrival']].agg(' - '.join, axis=1))
      )

In [19]:
ts_visualisation(data=df,
                 list_id=df.groupby('id').n_train.sum()
                           .sort_values(ascending=False).index[0:20],
                 group='id',
                 x_var='date',
                 y_var=['n_train', 'n_train_pred'],
                 col_var=['black', 'red'],
                 threshold_train=THRESHOLD_TRAIN,
                 weekdays=True, scatter=False)